#### Import and pre-process data

In [45]:
import pandas as pd
from google.oauth2 import service_account
import gspread

SERVICE_ACCCOUNT_FILE = "google_key.json"

SHEET_ID = "YOUR_SHEET_ID"
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCCOUNT_FILE, scopes=["https://www.googleapis.com/auth/spreadsheets"])
client = gspread.authorize(credentials)

spreadsheet = client.open_by_key(SHEET_ID)
worksheet = spreadsheet.get_worksheet(0)
data = pd.DataFrame(worksheet.get_all_records())


In [ ]:
from faker import Faker

data["MONTH"] = pd.to_datetime(data["MONTH"])
data["FACEBOOK_SPEND"] = pd.to_numeric(data["FACEBOOK_SPEND"])
data["PBA_SPEND"] = pd.to_numeric(data["PBA_SPEND"])
data["TOTAL_IMAGE_TEMPLATE_SPEND"] = pd.to_numeric(data["TOTAL_IMAGE_TEMPLATE_SPEND"])
data["CUSTOMER_CHURN_MONTH"] = pd.to_datetime(data["CUSTOMER_CHURN_MONTH"])
data["DYNAMIC_ADS_IMAGE_TEMPLATE_SPEND"] = pd.to_numeric(
    data["DYNAMIC_ADS_IMAGE_TEMPLATE_SPEND"]
)
data["AUTOMATED_ADS_SPEND"] = pd.to_numeric(data["AUTOMATED_ADS_SPEND"])
data["CUSTOMER_AGE"] = pd.to_numeric(data["CUSTOMER_AGE"])
data["HOURS_SPENT_SUPPORT"] = pd.to_numeric(data["HOURS_SPENT_SUPPORT"])


data["AUTOMATED_ADS_SPEND_PERC"] = (
    100 * data["AUTOMATED_ADS_SPEND"] / data["FACEBOOK_SPEND"]
)
data["PBA_SPEND_PERC"] = 100 * data["PBA_SPEND"] / data["FACEBOOK_SPEND"]

data["TOTAL_IMAGE_TEMPLATE_SPEND_PERC"] = (
    100 * data["TOTAL_IMAGE_TEMPLATE_SPEND"] / data["FACEBOOK_SPEND"]
)

data["DYNAMIC_ADS_IMAGE_TEMPLATE_SPEND_PERC"] = (
    100 * data["DYNAMIC_ADS_IMAGE_TEMPLATE_SPEND"] / data["FACEBOOK_SPEND"]
)

data["CHURNED"] = (~data["CUSTOMER_CHURN_MONTH"].isna()).astype(int)

data_processed = data.drop(
    columns=[
        "AUTOMATED_ADS_SPEND",
        "PBA_SPEND",
        "TOTAL_IMAGE_TEMPLATE_SPEND",
        "DYNAMIC_ADS_IMAGE_TEMPLATE_SPEND",
    ]
)

fake = Faker()
fake_names_table = pd.DataFrame(columns=["CUSTOMER", "FAKE_NAME"])
fake_names_table["CUSTOMER"] = data["CUSTOMER"].unique()
fake_names = []
for i in range(fake_names_table.shape[0]):
    fake_names.append(f"{fake.company()}_{i}")
fake_names_table["FAKE_NAME"] = pd.Series(fake_names)

data_processed = data_processed.merge(fake_names_table, how="left", on="CUSTOMER")

display(data_processed.head(10))
print(f"{data_processed.shape[0]} rows, {data_processed.shape[1]} columns")

#### How has spend attributed to different features changed over time? Which features perform the best and the worst? What is the overall Facebook spend over time?

In [ ]:
mean_spend_over_time = (
    data_processed.groupby(by=["MONTH"])
    .agg(
        {
            "AUTOMATED_ADS_SPEND_PERC": ["mean"],
            "PBA_SPEND_PERC": ["mean"],
            "TOTAL_IMAGE_TEMPLATE_SPEND_PERC": ["mean"],
            "DYNAMIC_ADS_IMAGE_TEMPLATE_SPEND_PERC": ["mean"],
            "FACEBOOK_SPEND": ["mean"],
        }
    )
    .sort_values(by=["MONTH"], ascending=True)
)

mean_spend_over_time.columns = mean_spend_over_time.columns.map("_".join)
mean_spend_over_time.reset_index(inplace=True)

mean_spend_over_time_w_churn = (
    data_processed.groupby(by=["MONTH", "CHURNED"])
    .agg(
        {
            "AUTOMATED_ADS_SPEND_PERC": ["mean"],
            "PBA_SPEND_PERC": ["mean"],
            "TOTAL_IMAGE_TEMPLATE_SPEND_PERC": ["mean"],
            "DYNAMIC_ADS_IMAGE_TEMPLATE_SPEND_PERC": ["mean"],
            "FACEBOOK_SPEND": ["mean"],
        }
    )
    .sort_values(by=["MONTH"], ascending=True)
)

mean_spend_over_time_w_churn.columns = mean_spend_over_time_w_churn.columns.map(
    "_".join
)

mean_spend_over_time_w_churn.reset_index(inplace=True)

mean_spend_over_time_churned = mean_spend_over_time_w_churn.loc[
    mean_spend_over_time_w_churn["CHURNED"] == 1, :
]
mean_spend_over_time_not_churned = mean_spend_over_time_w_churn.loc[
    mean_spend_over_time_w_churn["CHURNED"] == 0, :
]

In [ ]:
from matplotlib import pyplot as plt


fig, (ax, ax1, ax2, ax3) = plt.subplots(1, 4)
fig.set_figheight(7)
fig.set_figwidth(25)
# fig.suptitle(f"spends")
fig.autofmt_xdate(rotation=45)

y_labels = mean_spend_over_time.columns.drop(["MONTH", "FACEBOOK_SPEND_mean"])

ax.plot(mean_spend_over_time["MONTH"], mean_spend_over_time[y_labels], label=y_labels)
ax.set_title("Trend of spends per feature, overall")
ax.set_ylabel(r"Feature spend as % of Facebook spend")
ax.set_xlabel("Date")
ax.legend(bbox_to_anchor=(0.5, 1.25), loc="upper center", ncol=1, prop={"size": 6})

ax1.plot(
    mean_spend_over_time_churned["MONTH"],
    mean_spend_over_time_churned[y_labels],
    label=y_labels,
)
ax1.set_title("Trend of spends per feature, churned")
ax1.set_ylabel(r"Feature spend as % of Facebook spend")
ax1.set_xlabel("Date")
ax1.legend(bbox_to_anchor=(0.5, 1.25), loc="upper center", ncol=1, prop={"size": 6})

ax2.plot(
    mean_spend_over_time_not_churned["MONTH"],
    mean_spend_over_time_not_churned[y_labels],
    label=y_labels,
)
ax2.set_title("Trend of spends per feature, not churned")
ax2.set_ylabel(r"Feature spend as % of Facebook spend")
ax2.set_xlabel("Date")
ax2.legend(bbox_to_anchor=(0.5, 1.25), loc="upper center", ncol=1, prop={"size": 6})


ax3.plot(
    mean_spend_over_time["MONTH"],
    mean_spend_over_time["FACEBOOK_SPEND_mean"],
    label="FACEBOOK_SPEND_mean",
)
ax3.set_title("Facebook spend, overall")
ax3.set_ylabel("Spend [Unknown units]")
ax3.set_xlabel("Date")
ax3.legend(bbox_to_anchor=(0.5, 1.25), loc="upper center", ncol=1, prop={"size": 6})

####  How many customers generate 80% of spend? What are the top 5 most valuable customers in terms of Facebook spend vs hours spent on support? 

In [ ]:
total_customer_spend = data_processed.groupby(by=["FAKE_NAME"]).agg(
    {"FACEBOOK_SPEND": ["sum"], 
     "HOURS_SPENT_SUPPORT": ["sum"], 
     "CUSTOMER_AGE": ["max"]
    }
)
total_customer_spend.columns = total_customer_spend.columns.map("_".join)
total_customer_spend.reset_index(inplace=True)
total_customer_spend.sort_values(by="FACEBOOK_SPEND_sum", ascending=False, inplace=True)

In [ ]:
quantile_08 = total_customer_spend["FACEBOOK_SPEND_sum"].quantile([0.8]).values[0]
percentage_of_spend_80_percentile = (
    total_customer_spend.loc[
        total_customer_spend["FACEBOOK_SPEND_sum"] > quantile_08, :
    ]["FACEBOOK_SPEND_sum"].sum()
) / (total_customer_spend["FACEBOOK_SPEND_sum"].sum())

number_of_customers_80_percentile = total_customer_spend.loc[
    total_customer_spend["FACEBOOK_SPEND_sum"] > quantile_08, :
].shape[0]

print(
    f"{number_of_customers_80_percentile} customers (20% of all customers) generate {100*percentage_of_spend_80_percentile}% of Facebook spend"
)

In [ ]:
period_start = data_processed["MONTH"].min()
period_end = data_processed["MONTH"].max()


fig, (ax, ax1, ax2) = plt.subplots(1, 3)
fig.set_figheight(7)
fig.set_figwidth(20)
# fig.suptitle("")
fig.autofmt_xdate(rotation=45)


ax.bar(
    total_customer_spend.iloc[0:9, 0],
    total_customer_spend.iloc[0:9, 1],
)
ax.set_xlabel("Customer ID")
ax.set_ylabel("Spend [Units unknown]")
ax.set_title(f"Top 10 customers' by Facebook spend in the considered period")

hours_threshold = 400
spend_threshold = 1e7
high_value_customers = total_customer_spend.loc[
    (total_customer_spend["FACEBOOK_SPEND_sum"] > spend_threshold)
    & (total_customer_spend["HOURS_SPENT_SUPPORT_sum"] < hours_threshold),
    :,
].sort_values(by=["FACEBOOK_SPEND_sum", "HOURS_SPENT_SUPPORT_sum"], ascending=False)
low_value_customers = total_customer_spend.loc[
    (total_customer_spend["FACEBOOK_SPEND_sum"] < spend_threshold)
    | (total_customer_spend["HOURS_SPENT_SUPPORT_sum"] > hours_threshold),
    :,
]

ax1.plot(
    high_value_customers.iloc[:, 2],
    high_value_customers.iloc[:, 1],
    "g^",
    low_value_customers.iloc[:, 2],
    low_value_customers.iloc[:, 1],
    "r^",
)
ax1.set_xlabel("Support effort [hours]")
ax1.set_ylabel("Spend [Units unknown]")
ax1.set_title(f"Facebook spend vs Support hours in the considered period")
# ax1.set_yscale("log")

for row in range(5):
    ax1.annotate(
        high_value_customers.iloc[row, 0],
        (high_value_customers.iloc[row, 2], high_value_customers.iloc[row, 1]),
    )

age_threshold = 60
spend_threshold = 1e7
high_loyalty_customers = total_customer_spend.loc[
    (total_customer_spend["FACEBOOK_SPEND_sum"] > spend_threshold)
    & (total_customer_spend["CUSTOMER_AGE_max"] > age_threshold),
    :,
].sort_values(by=["FACEBOOK_SPEND_sum", "CUSTOMER_AGE_max"], ascending=False)
low_loyalty_customers = total_customer_spend.loc[
    (total_customer_spend["FACEBOOK_SPEND_sum"] < spend_threshold)
    | (total_customer_spend["CUSTOMER_AGE_max"] < age_threshold),
    :,
]


ax2.plot(
    high_loyalty_customers.iloc[:, 3],
    high_loyalty_customers.iloc[:, 1],
    "g^",
    low_loyalty_customers.iloc[:, 3],
    low_loyalty_customers.iloc[:, 1],
    "r^",
)
ax2.set_xlabel("Customer age [months]")
ax2.set_ylabel("Spend [Units unknown]")
ax2.set_title(f"Facebook spend vs Support hours in the considered period")
# ax2.set_yscale("log")

for row in range(5):
    ax2.annotate(
        high_loyalty_customers.iloc[row, 0],
        (high_loyalty_customers.iloc[row, 3], high_loyalty_customers.iloc[row, 1]),
    )

#### What is the churn rate over time? What is the sutomer age?



In [ ]:
months = pd.Series(data_processed["MONTH"].unique()).sort_values(ascending=True)

customers_at_month_start = set()
churn_rate = pd.DataFrame(columns=["MONTH", "CHURN_RATE"])
number_of_customers = pd.DataFrame(columns=["MONTH", "CUSTOMER_COUNT"])
for month in months:
    month_data = data_processed.loc[
        data_processed["MONTH"] == month, ["FAKE_NAME", "CHURNED"]
    ]
    customers_this_month = set(month_data["FAKE_NAME"].unique())
    customers_at_month_start = customers_at_month_start.union(customers_this_month)
    churned_customers = set(
        month_data.loc[month_data["CHURNED"] == 1, "FAKE_NAME"].unique()
    )
    churn_rate_month = pd.DataFrame(
        data={
            "MONTH": [month],
            "CHURN_RATE": [
                100 * len(churned_customers) / len(customers_at_month_start)
            ],
        }
    )
    churn_rate = pd.concat([churn_rate, churn_rate_month])
    customers_at_month_start.difference_update(churned_customers)
    customers_count = pd.DataFrame(
        data={"MONTH": [month], "CUSTOMER_COUNT": [len(customers_at_month_start)]}
    )
    number_of_customers = pd.concat([number_of_customers, customers_count])

In [ ]:
customer_age = data_processed.groupby(by=["FAKE_NAME", "CHURNED"]).agg(
    {
        "CUSTOMER_AGE": ["max"],
    }
)
customer_age.columns = customer_age.columns.map("_".join)
customer_age.reset_index(inplace=True)
customer_age.sort_values(by="CUSTOMER_AGE_max", ascending=False, inplace=True)

In [ ]:
fig, (ax, ax1, ax2) = plt.subplots(1, 3)
fig.set_figheight(7)
fig.set_figwidth(21)
# fig.suptitle("")
fig.autofmt_xdate(rotation=45)


ax.plot(churn_rate["MONTH"], churn_rate["CHURN_RATE"])
ax.set_xlabel("Month")
ax.set_ylabel("Churn rate [%]")
ax.set_title(f"Churn rate over time")

ax1.hist(customer_age.loc[:, "CUSTOMER_AGE_max"])
ax1.set_xlabel("Customer age [months]")
ax1.set_ylabel("Customer count")
ax1.set_title(f"Distribution of customer age")

ax2.plot(number_of_customers["MONTH"], number_of_customers["CUSTOMER_COUNT"])
ax2.set_xlabel("Month")
ax2.set_ylabel("Customer count")
ax2.set_title(f"NUmber of customers over time")

plt.show()